<a href="https://colab.research.google.com/github/yeonui-0626/topic-modeling/blob/main/LDA_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 0. 패키지 로드
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [3]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
import spacy

In [7]:
!pip install pyLDAvis

In [11]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
'''
LDA는  각 문서를 특정 비율의 주제 모음으로 간주한다.
각 주제는 키워드의 모음으로 다시 일정 비율로 표시된다.
알고리즘에 주제 수를 제공하면서 문서 내 주제 분포와 주제 내 키워드 분포를 재배열하여
주제-키워드 분포의 좋은 구성을 얻을 수 있다.

토픽은 키워드의 집합체이다. 키워드만 봐도 주제가 무엇인지 알 수 있다.

좋은 주제를 분류하지 위한 핵심요소는 아래와 같다.
1.텍스크 처리 품질
2. 텍스트가 이야기하는 다양한 주제.
3. 토픽 모델린 알고리즘의 선택
4. 알고리즘에 제공되는 주제의 수
5. 알고리즘 조정 매개변수 
'''


In [168]:
# 1. 불용어 준비
stop_word=['필요','요구','흥미','성격','능력','사람','때문', '관련', '담당','경우','사항','불편','이외']

In [142]:
# 2. 데이터 로드
# data_load = pd.read_csv('/content/drive/MyDrive/alba-aptitude.csv', encoding='cp949')
# data_load.head()

data=[]

for idx, row in data_load.iterrows():
  if idx == 9 :
     continue
  data.append(row['context'])

data

['조리사 및 주방장은 예민한 미각이 필요하며 새로운 음식 메뉴를 개발하기 위해 진취적이고 창의적인 사람에게 적합하다. 오랜 시간 서서 일해야 하기 때문에 이를 견딜 수 있는 인내심과 끈기, 체력이 요구된다. 주방에서 여럿이 같이 조리하는 경우가 많아 협동심이 필요하고, 또 고객에 대한 서비스 정신이 필요하다. 예술형과 탐구형의 흥미를 가진 사람에게 적합하다.무엇보다 설거지, 청소, 음식재료 다듬기 등의 힘든 일을 잘 견디어낼 수 있는 인내심과 끈기, 강인한 체력이 요구된다. 평소 음식 만들기를 좋아하고 미각이 발달한 사람에게 적합하다. 현실형과 예술형의 흥미를 가진 사람에게 적합하며, 정직, 남에 대한 배려, 신뢰성, 적응력 등의 성격을 가진 사람들에게 유리하다.제품이나 서비스의 대한 정확한 가격을 파악하여 고객에게 정확하고 신속하게 금액을 징수하고 거스름돈을 준비할 수 있는 계산능력이 요구된다. 고객에 대한 서비스 정신과 적극성, 친절함 등이 요구된다. 관습형과 현실형의 흥미를 가진 사람에게 적합하며, 정직, 남에 대한 배려, 사회성 등의 성격을 가진 사람들에게 유리하다.',
 '각종 와인의 종류와 맛, 음식과의 궁합에 대해 알고 있어야 하며, 이를 위해 포도의 품종, 숙성방법, 원산지, 수확연도 등 와인의 특징과 보관방법에 대한 풍부한 지식이 요구된다. 와인을 서비스하는데 필요한 기본적인 기물이나 장비들을 자유롭게 사용할 수 있어야 하며, 고객이 편안하게 와인을 즐길 수 있도록 하는 고객 서비스 정신이 요구된다. 와인을 좋아해야하며, 계속해서 변화하는 와인의 세계적 흐름을 파악하고 이에 대해 공부하는 자세가 필요하다.조리사 및 주방장은 예민한 미각이 필요하며 새로운 음식 메뉴를 개발하기 위해 진취적이고 창의적인 사람에게 적합하다. 오랜 시간 서서 일해야 하기 때문에 이를 견딜 수 있는 인내심과 끈기, 체력이 요구된다. 주방에서 여럿이 같이 조리하는 경우가 많아 협동심이 필요하고, 또 고객에 대한 서비스 정신이 필요하다. 예술형과 탐구형의 흥미를 가진 사람

In [120]:
# 3. 단어 토큰화(벡터화)
!pip install konlpy
from konlpy.tag import Komoran
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

tagger = Komoran()
def get_nouns(text):
    nouns = tagger.nouns(text)
    return [n for n in nouns if len(n) > 1]


vectorizer = CountVectorizer(analyzer='word', tokenizer=get_nouns, min_df=0,
                                    stop_words = stop_word, ngram_range=(1,1))

tdm = vectorizer.fit_transform(data)

print("(문서 수, 단어 수)")
print(tdm.shape) # (문서 수, 단어 수)

(문서 수, 단어 수)
(149, 1189)


In [169]:
documents=[]
for i in range(len(data)):
  nouns = get_nouns(data[i])
  doc_word=[]
  for token in nouns:
    if token not in stop_word:
      doc_word.append(token)
  documents.append(doc_word)
len(documents)

149

In [185]:
# 4. 말뭉치 생성(id2word, corpus)
# id2word : 단어에 id값을 부여한 dictionary
# corpus : id별(단어별) 빈도수

id2word = corpora.Dictionary(documents)
corpus = [id2word.doc2bow(document) for document in documents]
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 3),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 1),
  (9, 2),
  (10, 1),
  (11, 2),
  (12, 1),
  (13, 3),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 3),
  (22, 2),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 2),
  (27, 2),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 2),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 2),
  (44, 1)],
 [(1, 1),
  (4, 5),
  (5, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (11, 1),
  (12, 2),
  (13, 2),
  (14, 1),
  (16, 1),
  (19, 1),
  (20, 1),
  (21, 4),
  (22, 1),
  (24, 1),
  (25, 1),
  (26, 3),
  (29, 1),
  (30, 1),
  (31, 2),
  (32, 1),
  (34, 1),
  (36, 1),
  (38, 2),
  (40, 1),
  (41, 1),
  (44, 1),
  (45, 1),
  (46, 2),
  (47, 1),
  (48, 2),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 2),
  (54, 2),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 2),
  (59, 2),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (

In [173]:
id2word.token2id

{'가격': 0,
 '개발': 1,
 '거스름돈': 2,
 '계산': 3,
 '고객': 4,
 '관습': 5,
 '금액': 6,
 '끈기': 7,
 '메뉴': 8,
 '미각': 9,
 '발달': 10,
 '배려': 11,
 '사회': 12,
 '서비스': 13,
 '서서': 14,
 '설거지': 15,
 '시간': 16,
 '신뢰': 17,
 '신속': 18,
 '여럿': 19,
 '예술': 20,
 '음식': 21,
 '인내심': 22,
 '재료': 23,
 '적극': 24,
 '적응력': 25,
 '정신': 26,
 '정직': 27,
 '제품': 28,
 '조리': 29,
 '조리사': 30,
 '주방': 31,
 '주방장': 32,
 '준비': 33,
 '진취': 34,
 '징수': 35,
 '창의': 36,
 '청소': 37,
 '체력': 38,
 '친절': 39,
 '탐구': 40,
 '파악': 41,
 '평소': 42,
 '현실': 43,
 '협동심': 44,
 '각종': 45,
 '감각': 46,
 '계속': 47,
 '고객 서비스': 48,
 '공부': 49,
 '궁합': 50,
 '그릇': 51,
 '기물': 52,
 '기본': 53,
 '대처': 54,
 '동안': 55,
 '매너': 56,
 '물리': 57,
 '방법': 58,
 '변화': 59,
 '보관': 60,
 '사용': 61,
 '상황': 62,
 '색채': 63,
 '세계': 64,
 '소품': 65,
 '수확': 66,
 '숙성': 67,
 '식사': 68,
 '아이디어': 69,
 '안목': 70,
 '연구': 71,
 '연도': 72,
 '와인': 73,
 '요리': 74,
 '원산지': 75,
 '융통': 76,
 '응대': 77,
 '이해': 78,
 '자세': 79,
 '자유': 80,
 '장비': 81,
 '장시간': 82,
 '장식': 83,
 '전달': 84,
 '전문': 85,
 '접시': 86,
 '정리': 87,
 '정보': 88,
 '제시': 89,
 '조

In [ ]:
# ------ Vectorizer 사용한 거 ---------- #
# 4. 말뭉치 생성(id2word, corpus)
# id2word : 단어에 id값을 부여한 dictionary
# corpus : id별(단어별) 빈도수

# words = vectorizer.get_feature_names()  #단어 목록 저장
# id2word = corpora.Dictionary([words])

# count = tdm.sum(axis=0) #열별로 모든 행 합계 -> 단어별로 빈도 구함
# corpus = list(zip(id2word.token2id.values(), count.flat))
# corpus

In [186]:
# 5. 토픽 모델 생성
'''
update_every : 모델 매개변수를 업데이트 해야하는 빈도
chunksize : 한번 학습할 때 문서를 얼마나 볼 지
passes : 모델 학습시 전체 코퍼스에서 모델을 학습시키는 빈도제어
 - epochs와 동일(모델을 학습시키는 횟수)
alpha : 
per_word_topics : 
'''
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=16, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [187]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics(num_words=30))
doc_lda = lda_model[corpus]

[(0,
  '0.068*"분석" + 0.052*"사고" + 0.038*"논리" + 0.031*"해결" + 0.025*"지식" + 0.024*"탐구" '
  '+ 0.024*"문제" + 0.022*"제품" + 0.021*"개발" + 0.019*"상품" + 0.018*"판단력" + '
  '0.018*"분석력" + 0.017*"이해" + 0.015*"호기심" + 0.012*"프로그램" + 0.012*"컴퓨터" + '
  '0.012*"정신" + 0.012*"혁신" + 0.012*"창의성" + 0.011*"작성" + 0.011*"진행" + '
  '0.011*"고객" + 0.010*"마케팅" + 0.010*"진취" + 0.010*"기술자" + 0.009*"영업" + '
  '0.009*"장시간" + 0.008*"소질" + 0.008*"창의" + 0.008*"원활"'),
 (1,
  '0.028*"예술" + 0.025*"기술" + 0.023*"지식" + 0.020*"감각" + 0.018*"장비" + 0.016*"탐구" '
  '+ 0.015*"리더십" + 0.014*"관계" + 0.013*"대인" + 0.013*"사회" + 0.013*"대처" + '
  '0.012*"체력" + 0.012*"방송" + 0.012*"끈기" + 0.011*"인내심" + 0.011*"진취" + '
  '0.011*"창의력" + 0.011*"판단력" + 0.011*"작업" + 0.010*"소리" + 0.010*"촬영" + '
  '0.009*"책임감" + 0.009*"연기" + 0.008*"원활" + 0.008*"무대" + 0.008*"문제" + '
  '0.008*"혁신" + 0.008*"제작" + 0.008*"활용" + 0.008*"인간관계"'),
 (2,
  '0.053*"학생" + 0.032*"사회" + 0.031*"교육" + 0.030*"구사" + 0.030*"책임감" + '
  '0.025*"이론" + 0.023*"교육자" + 0.022*"언어" + 0.021*"상황" + 0.0

In [189]:
# 모델 복잡도 및 일관성 점수 계산

# 복잡도(Perplexity)
print('Perplexity : ',lda_model.log_perplexity(corpus))

# 일관성(Coherence)
coherence_model_lda  = CoherenceModel(model=lda_model, texts=documents, dictionary=id2word,coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

Perplexity :  -6.193631321328466
Coherence Score:  0.40259392347893974
